# fileset library usage guides
Snowflake Python FileSet library is one of the Snowflake ML tools. It provides easy approaches to load large data
from Snowflake to where you Python code runs. It includes two components: Snowflake Filesystem and Snowflake FileSet.

## Getting started
Snowflake Filesystem/FileSet requires either a
[Snowflake Python connection](https://docs.snowflake.com/en/user-guide/python-connector.html) or
[Snowpark seesion](https://docs.snowflake.com/en/developer-guide/snowpark/python/index.html).

In [6]:
from snowflake import connector
from snowflake.ml.utils import connection_params
from snowflake.snowpark import Session

# We are using sfctest0 account.
connection_parameters = connection_params.SnowflakeLoginOptions("sfctest0")
snowpark_session = Session.builder.configs(connection_parameters).create()
sf_connection = connector.connect(**connection_parameters)

## Snowflake Filesystem APIs

We provide an implementation of `fsspec.AbstractFileSystem` which enables a filesystem-alike experience for
accessing files in an __internal, server-side encrypted__ Snowflake stage.
See the Snowflake [documentation](https://docs.snowflake.com/en/sql-reference/sql/create-stage#internal-stage-parameters-internalstageparams)
on how to create internal server-side encrypted stages.

### Create a new Snowflake Filesystem object
The Snowflake Filesystem object can be created by either a Snowflake python connection or Snowpark session:

In [27]:
import fsspec

# sfsfs module is required to register "sfc" protocol to fsspec.
from snowflake.ml.fileset import sfcfs

# Create a fs object with snowflake python connection
sffs1 = sfcfs.SFFileSystem(sf_connection=sf_connection)
# Create a fs object with snowpark session
sffs2 = sfcfs.SFFileSystem(snowpark_session=snowpark_session)

# Create a fs object with snowflake python connection via fssepc interface
sffs3 = fsspec.filesystem("sfc", sf_connection=sf_connection)
# Create a fs object with snowpark session via fssepc interface
sffs4 = fsspec.filesystem("sfc", snowpark_session=snowpark_session)


Some nice feature of fsspec is also inherited by Snowflake Filesystem. For example, you can set the memory cache type and buffer size. You can also utilize the local cache feature of fsspec:

In [28]:
local_cache_path = "/tmp/files/"
cached_fs = fsspec.filesystem(
    "filecache",
    target_protocol="sfc",
    target_options={"sf_connection": sf_connection, "cache_types": "bytes", "block_size": 32 * 2**20},
    cache_storage=local_cache_path,
)

### List files in a stage
The Snowflake Filesystem can list stage files under a directory in the format of `@<database>.<schema>.<stage>/<filepath>`.

In [29]:
cached_fs.ls("@ML_DATASETS.public.zhuo_models/zpeng_predict/")

['@ML_DATASETS.public.zhuo_models/zpeng_predict/serving_udf.py',
 '@ML_DATASETS.public.zhuo_models/zpeng_predict/udf_py_1679730148.zip',
 '@ML_DATASETS.public.zhuo_models/zpeng_predict/udf_py_1779181627.zip',
 '@ML_DATASETS.public.zhuo_models/zpeng_predict/udf_py_184582763.zip',
 '@ML_DATASETS.public.zhuo_models/zpeng_predict/udf_py_261557963.zip',
 '@ML_DATASETS.public.zhuo_models/zpeng_predict/udf_py_392770850.zip',
 '@ML_DATASETS.public.zhuo_models/zpeng_predict/udf_py_735600790.zip',
 '@ML_DATASETS.public.zhuo_models/zpeng_predict/udf_py_73754703.zip']

In [30]:
sffs1.ls("@ML_DATASETS.public.zzhu_sse", detail=True)

[{'name': '@ML_DATASETS.public.zzhu_sse/test_data/',
  'size': 0,
  'type': 'directory',
  'md5': None,
  'last_modified': None},
 {'name': '@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz',
  'size': 3185489,
  'type': 'file',
  'md5': '61e772df25aea9e527b4238b61dacc0c',
  'last_modified': 'Wed, 1 Feb 2023 21:04:48 GMT'},
 {'name': '@ML_DATASETS.public.zzhu_sse/zzhu_test_cast_0_0_0.snappy.parquet',
  'size': 1207,
  'type': 'file',
  'md5': '50348586e24979d40e448315e8ffa23d',
  'last_modified': 'Wed, 11 Jan 2023 21:36:06 GMT'},
 {'name': '@ML_DATASETS.public.zzhu_sse/zzhu_test_nocast_0_0_0.snappy.parquet',
  'size': 1024,
  'type': 'file',
  'md5': '7c397f20481dd53d514f7438a1fe2f47',
  'last_modified': 'Wed, 11 Jan 2023 20:17:25 GMT'}]

### Open and read files
You can open a stage file in read mode:

In [31]:
with sffs1.open('@ML_DATASETS.public.zzhu_sse/test_data/data_7_7_3.snappy.parquet', mode='rb') as f:
    print(f.read(16))

b"PAR1\x15\x04\x15\xc0\xae'\x15\xa2\xa2\x0cL\x15"


Reading a file can also be done by using fsspec interface:

In [32]:
with fsspec.open("sfc://@ML_DATASETS.public.zzhu_sse/test_data/data_7_7_3.snappy.parquet", mode='rb', sf_connection=sf_connection) as f:
    print(f.read(16))

b"PAR1\x15\x04\x15\xc0\xae'\x15\xa2\xa2\x0cL\x15"


It's easy to integrate file reading with any plugins that are compatible with fsspec. For example, we can directly read a parquet stage file with pyarrow:

In [33]:
import pyarrow.parquet as pq

table = pq.read_table("sfc://@ML_DATASETS.public.zzhu_sse/test_data/data_7_7_3.snappy.parquet", filesystem=sffs1)
table.take([1, 3])

pyarrow.Table
_COL_0: decimal128(38, 0) not null
_COL_1: decimal128(38, 0) not null
_COL_2: double
_COL_3: decimal128(38, 0) not null
_COL_4: double
_COL_5: double
_COL_6: double
_COL_7: double
_COL_8: double
_COL_9: double
_COL_10: double
_COL_11: double
_COL_12: double
_COL_13: double
_COL_14: double
_COL_15: decimal128(38, 0) not null
_COL_16: decimal128(38, 0) not null
_COL_17: decimal128(38, 0)
_COL_18: decimal128(38, 0)
_COL_19: decimal128(38, 0) not null
_COL_20: decimal128(38, 0)
_COL_21: decimal128(38, 0) not null
_COL_22: decimal128(38, 0) not null
_COL_23: decimal128(38, 0) not null
_COL_24: decimal128(38, 0) not null
_COL_25: decimal128(38, 0) not null
_COL_26: decimal128(38, 0)
_COL_27: decimal128(38, 0) not null
_COL_28: decimal128(38, 0) not null
_COL_29: decimal128(38, 0) not null
_COL_30: decimal128(38, 0)
_COL_31: decimal128(38, 0) not null
_COL_32: decimal128(38, 0) not null
_COL_33: decimal128(38, 0)
_COL_34: decimal128(38, 0)
_COL_35: decimal128(38, 0)
_COL_36: dec

We can also read other file formats like:

In [34]:
import gzip

with cached_fs.open("sfc://@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz", mode='rb', sf_connection=sf_connection) as f:
    f = gzip.GzipFile(fileobj=f)
    for i in range(3):
        print(f.readline())


b'Sentence #,Word,POS,Tag\r\n'
b'Sentence: 1,Thousands,NNS,O\r\n'
b',of,IN,O\r\n'


With fsspec's native support of local caching, a local copy of the csv file will be created. We can even directly read the local copy. 

In [35]:
# Check the local cache
cache_name = cached_fs.hash_name("@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz", False)
with open(f'{local_cache_path}{cache_name}', mode='rb') as f:
    f = gzip.GzipFile(fileobj=f)
    for i in range(3):
        print(f.readline())

b'Sentence #,Word,POS,Tag\r\n'
b'Sentence: 1,Thousands,NNS,O\r\n'
b',of,IN,O\r\n'


### Other supported methods
Snowflake filesystem supports most read-only methods supported by fsspec, which includes `find()`, `info()`,
`isdir()`, `isfile()`, `exists()`, and so on.

In [36]:
print(sffs1.info("@ML_DATASETS.public.zzhu_sse/"))
print(sffs1.info("@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz"))

{'name': '@ML_DATASETS.public.zzhu_sse/', 'size': 0, 'type': 'directory'}
{'name': '@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz', 'size': 3185489, 'type': 'file', 'md5': '61e772df25aea9e527b4238b61dacc0c', 'last_modified': 'Wed, 1 Feb 2023 21:04:48 GMT'}


In [37]:
print(sffs1.isdir("@ML_DATASETS.public.zzhu_sse"))
print(sffs1.isdir("@ML_DATASETS.public.zzhu_sse/"))
print(sffs1.isdir("@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz"))

True
True
False


In [38]:
print(sffs1.isfile("@ML_DATASETS.public.zzhu_sse"))
print(sffs1.isfile("@ML_DATASETS.public.zzhu_sse/"))
print(sffs1.isfile("@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz"))

False
False
True


In [39]:
print(sffs1.exists("@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz"))
print(sffs1.exists("@ML_DATASETS.public.zzhu_sse/random_name"))

True
False


In [40]:
for i in sffs1.walk("@ML_DATASETS.public.zhuo_models"):
    print(i)

('@ML_DATASETS.public.zhuo_models', ['00f9c83e76870f14cb5265de20ef8d6cfcc92f6c9e05065b317c6bcd7a479ad7', '184f3054e0dc1085e938ded26d18f47319441ac219866692a5e47fb9ee85ffbe', '36cbf84c124c90cc8c0f122b448d3089167b0f417dd8a2ecf4ef68ccb583b91d', '4cfdd792e91f96b0bd442971b9313877f9dcffde0be71f4009937e5d83d800f8', '8af58850e9d77444ce614c80a18bd051944f869956e24acf54db7863f30753ec', 'a766abf0b3bfc35026401af725d82e2873d50c8c062e62318970ef91eacab670', 'e04cf101c6f55cc320a5ef9a9758ee11fe927ecabda364a2afc49c1d4d3e285a', 'e60557aae588ebb41aa6447a70a528030d68a39b50affc445815b1cf69b8fa83', 'model_too_large', 'zpeng_predict', 'zpeng_predict_separated'], [])
('@ML_DATASETS.public.zhuo_models/00f9c83e76870f14cb5265de20ef8d6cfcc92f6c9e05065b317c6bcd7a479ad7', [], ['serving_model.pkl'])
('@ML_DATASETS.public.zhuo_models/184f3054e0dc1085e938ded26d18f47319441ac219866692a5e47fb9ee85ffbe', [], ['serving_model_ltitu.pkl'])
('@ML_DATASETS.public.zhuo_models/36cbf84c124c90cc8c0f122b448d3089167b0f417dd8a2ecf4ef68c

In [41]:
sffs1.find("@ML_DATASETS.public.zhuo_models")

['@ML_DATASETS.public.zhuo_models/00f9c83e76870f14cb5265de20ef8d6cfcc92f6c9e05065b317c6bcd7a479ad7/serving_model.pkl',
 '@ML_DATASETS.public.zhuo_models/184f3054e0dc1085e938ded26d18f47319441ac219866692a5e47fb9ee85ffbe/serving_model_ltitu.pkl',
 '@ML_DATASETS.public.zhuo_models/36cbf84c124c90cc8c0f122b448d3089167b0f417dd8a2ecf4ef68ccb583b91d/serving_model.pkl',
 '@ML_DATASETS.public.zhuo_models/4cfdd792e91f96b0bd442971b9313877f9dcffde0be71f4009937e5d83d800f8/serving_model.pkl',
 '@ML_DATASETS.public.zhuo_models/8af58850e9d77444ce614c80a18bd051944f869956e24acf54db7863f30753ec/serving_model.pkl',
 '@ML_DATASETS.public.zhuo_models/a766abf0b3bfc35026401af725d82e2873d50c8c062e62318970ef91eacab670/serving_model.pkl',
 '@ML_DATASETS.public.zhuo_models/e04cf101c6f55cc320a5ef9a9758ee11fe927ecabda364a2afc49c1d4d3e285a/serving_model_bohkt.pkl',
 '@ML_DATASETS.public.zhuo_models/e60557aae588ebb41aa6447a70a528030d68a39b50affc445815b1cf69b8fa83/serving_model_gaphw.pkl',
 '@ML_DATASETS.public.zhuo_mod

In [42]:
sffs1.du("@ML_DATASETS.public.zhuo_models/zpeng_predict")

828900944

In [43]:
sffs1.glob("@ML_DATASETS.public.zhuo_models/zpeng_predict/*_udf*")

['@ML_DATASETS.public.zhuo_models/zpeng_predict/serving_udf.py']

## Snowflake FileSet APIs
A Snowflake `FileSet` represents an immutable snapshot of the result of a SQL query in the form of files materialized in
an internal server-side encrypted stage. It is built to make user's life easier when do machine learning tasks.

### Create a new FileSet object
A `FileSet` object can be created with either a Snowflake Python connection, or a Snowpark dataframe. You must also provide
an existing internal server-side encrypted stage, or a sub-directory under such a stage for `FileSet` to materialize
the data into.
See Snowflake [documentation](https://docs.snowflake.com/en/sql-reference/sql/create-stage#internal-stage-parameters-internalstageparams)
for how to create such a stage.

In [7]:
from snowflake.snowpark import functions
from snowflake.ml.fileset import fileset

query = "SELECT * FROM MYDATA LIMIT 5000000"

# stage "@ML_DATASETS.public.zhhu_sse" was created using
# CREATE STAGE ML_DATASETS.PUBLIC.ZZHU_SSE ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

# New FileSet with a Snowpark dataframe
df = snowpark_session.sql(query)
fileset_init_with_snowpark = fileset.FileSet.make(
    target_stage_loc="@ML_DATASETS.public.zzhu_sse/",
    name="init_with_sp",
    snowpark_dataframe=df,
    shuffle=False,
)

# New FileSet with a Snowflake Python connection
fileset_init_with_conn = fileset.FileSet.make(
    target_stage_loc="@ML_DATASETS.public.zzhu_sse/",
    name="init_with_conn",
    sf_connection=sf_connection,
    query=query,
    shuffle=True,
)

### Check the stage files inside a FileSet

In [45]:
fileset_init_with_snowpark.files()

['sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_005_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_105_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_205_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_305_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_405_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_505_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_605_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_607_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-

In [46]:
fileset_init_with_conn.files()

['sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_013_5_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_015_5_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_015_5_1.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_113_5_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_213_5_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_313_5_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_413_5_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_conn/data_01aa10fa-0405-a6ee-000c-a90105c77d73_513_5_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_c

### Feed Pytorch
Once you created a `FileSet`, you can get a torch `DataPipe` and give it to a torch `DataLoader`. The `DataLoader` iterates the FileSet data and yields batched torch Tensors.

In [9]:
from torch.utils.data import DataLoader

dp = fileset_init_with_snowpark.to_torch_datapipe(batch_size=4, shuffle=True, drop_last_batch=True)
for batch in DataLoader(dp, batch_size=None, num_workers=0):
    print(batch)
    break

{'__NULL_DASK_INDEX__': tensor([377144, 443658, 416818, 413535]), 'LABEL': tensor([1, 0, 1, 0]), 'I1': tensor([nan, nan, 3., 0.]), 'I2': tensor([-1, -1, -1, -1]), 'I3': tensor([nan, nan,  2., 14.]), 'I4': tensor([nan, nan, 0., 6.]), 'I5': tensor([3.3073e+04, 3.0190e+03, 5.0000e+00, 9.7570e+03]), 'I6': tensor([28.,  5.,  0., 55.]), 'I7': tensor([ 0.,  1.,  7., 27.]), 'I8': tensor([ 8.,  0.,  2., 19.]), 'I9': tensor([28.,  5., 62., 29.]), 'I10': tensor([nan, nan, 1., 0.]), 'I11': tensor([0., 1., 3., 3.]), 'I12': tensor([0., nan, nan, nan]), 'I13': tensor([nan, nan, 0., 6.]), 'C1': tensor([  98275684,   98275684, 1761418852,   98275684]), 'C2': tensor([  166103942,  -278296454,   166103942, -1595856491]), 'C3': tensor([ 2107754204, -1951961923,  1332335449,  -732442779]), 'C4': tensor([-2042314393,  -839758040, -1741218962,   144967799]), 'C5': tensor([ 633879704, 1135711049,  633879704, 1135711049]), 'C6': tensor([        nan,  2.1148e+09,  3.2621e+08, -7.2525e+07],
       dtype=torch.fl

# Feed TensorFlow

Similarily, you can get a `tf.data.Dataset` from a `FileSet`. Again, the `Dataset` dispenses batched TF Tensors.

In [13]:
import tensorflow as tf

ds = fileset_init_with_snowpark.to_tf_dataset(batch_size=4, shuffle=True, drop_last_batch=True)
assert(isinstance(ds, tf.data.Dataset))
for batch in ds:
    print(batch)
    break

{'__NULL_DASK_INDEX__': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([396667, 396684, 391010, 436069])>, 'LABEL': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>, 'I1': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1.,  6., nan,  0.], dtype=float32)>, 'I2': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([48,  3, -1,  1])>, 'I3': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([15., nan, nan,  1.], dtype=float32)>, 'I4': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([10.,  7., nan, nan], dtype=float32)>, 'I5': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([  444.,  1085., 33006.,  2617.], dtype=float32)>, 'I6': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([10., 43., 39.,  9.], dtype=float32)>, 'I7': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1., 14.,  1.,  3.], dtype=float32)>, 'I8': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([15., 10.,  7.,  2.], dtype=float32)>, 'I9': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([

### Delete FileSet
You can explicitly call `delete()` to delete the FileSet and its underlying stage. If it is not called, the stage will
be preseved, and you can recover it with the path to that stage.

In [48]:
fileset_init_with_conn.delete()

# The stage should get deleted in the stage.
print(sffs1.exists("@ML_DATASETS.public.zzhu_sse/init_with_conn"))

# The state of the FileSet will also be cleaned up
print(fileset_init_with_conn.files())

False
[]


### Recover existing data
If an old FileSet is not deleted, you can recover it in another Python program with its stage path and name:

In [49]:
# The FileSet recovering will check if all belonging files are generated with the same query id
recovered_fileset = fileset.FileSet(
    snowpark_session=snowpark_session,
    target_stage_loc="@ML_DATASETS.public.zzhu_sse",
    name="init_with_sp")

recovered_fileset.files()

['sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_005_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_105_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_205_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_305_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_405_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_505_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_605_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-0405-a6ee-000c-a90105c77d3f_607_0_0.snappy.parquet',
 'sfc://@ML_DATASETS.public.zzhu_sse/init_with_sp/data_01aa10fa-

In [50]:
recovered_fileset.delete()

# Now both FileSets are deleted
sffs1.ls("@ML_DATASETS.public.zzhu_sse/")

['@ML_DATASETS.public.zzhu_sse/test_data/',
 '@ML_DATASETS.public.zzhu_sse/ner_dataset.csv.gz',
 '@ML_DATASETS.public.zzhu_sse/zzhu_test_cast_0_0_0.snappy.parquet',
 '@ML_DATASETS.public.zzhu_sse/zzhu_test_nocast_0_0_0.snappy.parquet']